<a href="https://colab.research.google.com/github/ndajiya/wetrust-ui/blob/master/shrek_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U transformers datasets torch accelerate peft bitsandbytes sentencepiece
!pip install torch-pruning
!pip install git+https://github.com/huggingface/peft
!pip install einops

!apt-get update && apt-get install build-essential

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 74.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.4
    Uninstalling tokenizers-0.21.4:
      Successfully uninstalled tokenizers-0.21.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.55.4
    Uninstalling transformers-4.55.4:
      Successfully uninstalled transformers-4.55.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 3.9 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/peft to /tmp/pip-req-build-f6fh1hdm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft /tmp/pip-req-build-f6fh1hdm
  Resolved https://github.com/huggingface/peft to commit 2ea5

In [ ]:
files = ['shrek.txt' , 'shitposts.txt']

with open('merged.txt' , 'w') as outfile:
  for fname in files:
    try:
      with open(fname , 'r' , encoding = 'utf-8') as infile:
        outfile.write(infile.read() + "\n\n")
    except FileNotFoundError :
      print(f"[!] {fname} not found")
  print("text merged")

text merged


In [ ]:
from transformers import GPT2Tokenizer , GPT2LMHeadModel , Trainer , TrainingArguments
from datasets import load_dataset

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token =tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("gpt2")

dataset = load_dataset('text' , data_files = 'merged.txt')
def tokenize_function(examples) :
  return tokenizer(examples["text"] , truncation = True , padding = True , max_length = 128)

tokenized_datasets = dataset.map(tokenize_function, batched=True, num_proc=2)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Map (num_proc=2):   0%|          | 0/4813 [00:00<?, ? examples/s]

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="gpt2-shrek-chaos",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    save_steps=100,
    logging_steps=50,
    learning_rate=3e-4,
    weight_decay=0.01,
    fp16=True,
    logging_dir="logs",
    report_to="none",
    save_total_limit=1,

    disable_tqdm=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)

print("training..")
trainer.train()

model.save_pretrained("gpt2-shrek-chaos")
tokenizer.save_pretrained("gpt2-shrek-chaos")

print("saved to 'gpt2-shrek-chaos'")

training..


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.205100
100,2.826600
150,2.949900
200,2.776800
250,2.374700
300,2.577800
350,1.907300
400,1.935700
450,1.968000
500,2.010100


saved to 'gpt2-shrek-chaos'


In [ ]:
import torch
from transformers import GPT2LMHeadModel, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = GPT2LMHeadModel.from_pretrained(
    "gpt2-shrek-chaos",
    device_map="auto",
    quantization_config=bnb_config,
    dtype=torch.float16,
)


In [ ]:
import torch
import torch.nn as nn

model = GPT2LMHeadModel.from_pretrained("gpt2-shrek-chaos")
model.eval()
weighsToSparce = 0.5
for name, module in model.named_modules():
    if isinstance(module, nn.Linear):
        with torch.no_grad():
            w = module.weight.data
            mask = torch.rand(w.shape) > weighsToSparce
            w *= mask.to(w.device)

print("Model weights sparsified (" , weighsToSparce , ") zeroed out)")

model.save_pretrained("gpt2-shrek-chaos-sparse")

Model weights sparsified (50% zeroed out)


In [ ]:
!pip install mistral-common


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 119.0 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2-shrek-chaos")
tokenizer.save_pretrained("gpt2-shrek-chaos")

!rm -rf llama.cpp
!git clone https://github.com/ggerganov/llama.cpp
!pip install sentencepiece tokenizers gguf mistral-common

!cd llama.cpp && mkdir -p build && cd build && \
cmake .. \
-DCMAKE_BUILD_TYPE=Release \
&& make -j

!cp -r gpt2-shrek-chaos llama.cpp/models/gpt2-shrek-chaos

!cd llama.cpp && python3 convert_hf_to_gguf.py models/gpt2-shrek-chaos --outfile models/gpt2-shrek-chaos.gguf

!cd llama.cpp && ./build/bin/llama-quantize models/gpt2-shrek-chaos.gguf models/gpt2-shrek-chaos-Q2_K.gguf Q2_K

!ls -lh llama.cpp/models/*.gguf



Cloning into 'llama.cpp'...
remote: Enumerating objects: 60791, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 60791 (delta 86), reused 32 (delta 32), pack-reused 60669 (from 4)
Receiving objects: 100% (60791/60791), 151.29 MiB | 17.78 MiB/s, done.
Resolving deltas: 100% (44115/44115), done.
Updating files: 100% (1451/1451), done.
-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM comp

In [ ]:
!cd llama.cpp && ./build/bin/llama-cli -m models/gpt2-shrek-chaos-Q2_K.gguf -p "1+1 = " -n 100 --temp 1 --no-warmup --repeat_penalty 2.0

build: 6373 (0fce7a12) with cc (Ubuntu 11.4.0-1ubuntu1~22.04.2) 11.4.0 for x86_64-linux-gnu
main: llama backend init
main: load the model and apply lora adapter, if any
llama_model_loader: loaded meta data with 22 key-value pairs and 148 tensors from models/gpt2-shrek-chaos-Q2_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gpt2
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = Gpt2 Shrek Chaos
llama_model_loader: - kv   3:                         general.size_label str              = 124M
llama_model_loader: - kv   4:                           gpt2.block_count u32              = 12
llama_model_loader: - kv   5:                        gpt2.context_length u32              = 10